#### This is file to generate node features and edge index and features from Pandapower case file. The generated node features, edge index and edge features can be used to train GNN model.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pandapower as pp
from pandapower.networks import case118

/var/folders/0t/9z26630n32l3qw5w80zvvjqw0000gn/T/ipykernel_27030/2519735272.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
# Load any case file you need
net = case118()
net.gen.max_p_mw = 200.
# Run the case and see if there is any error
pp.rundcopp(net)

/opt/homebrew/lib/python3.11/site-packages/pandapower/io_utils.py:509: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(self.obj, precise_float=True, convert_axes=False, **self.d)
/opt/homebrew/lib/python3.11/site-packages/pandapower/io_utils.py:509: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(self.obj, precise_float=True, convert_axes=False, **self.d)
/opt/homebrew/lib/python3.11/site-packages/pandapower/io_utils.py:509: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(self.obj, precise_float=True, convert_axes=False, **self.d)
/opt/homebrew/lib/python3.11/site-packag

In [4]:
pp.to_excel(net, 'network/case118_original_network.xlsx')

ImportError: Some pandapower functionality use modules outside the setup.py requirements: to_excel() requires 'xlsxwriter'. 
xlsxwriter could not be imported.
To install all pandapower dependencies, pip install pandapower["all"] can be used.

In [5]:
# Change bus name from 'Name' to 'Bus_no'
net.bus.name.name = 'Bus_no'
# Bus no. starts from 0
net_index = net.bus.name.astype(int) - 1

#### Get node features from the case network

In [6]:
# Create DataFrame to store network data
network = pd.DataFrame()

# Set network index
network.index = net_index

In [7]:
# Get features from 'bus
network['vn_kv'] = net.bus.vn_kv.values

# Get features from 'load'
net.load.index = net.load.bus
network['load_p_mw'] = net.load.p_mw

# Get features from 'gen'
net.gen.index = net.gen.bus
network['max_gen_p_mw'] = net.gen.max_p_mw
network['min_gen_p_mw'] = net.gen.min_p_mw

# Get features from 'ext_grid' (ext_grid bus no: 68)
net.ext_grid.index = net.ext_grid.bus
network['max_gen_p_mw'][net.ext_grid.index.values] = net.ext_grid.max_p_mw
network['min_gen_p_mw'][net.ext_grid.index.values] = net.ext_grid.min_p_mw

# Get features from 'poly_cost'
# This needs manual work
poly_cost_index = pd.read_csv('./network/poly_cost_index.csv', index_col=0, header=0)
net.poly_cost.index = poly_cost_index.values.reshape((-1))
network['price_1'] = net.poly_cost.cp1_eur_per_mw
network['price_2'] = net.poly_cost.cp2_eur_per_mw2

# Additional features

network['x_pos'] = net.bus_geodata.x
network['y_pos'] = net.bus_geodata.y

/var/folders/0t/9z26630n32l3qw5w80zvvjqw0000gn/T/ipykernel_27030/111530940.py:15: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  network['max_gen_p_mw'][net.ext_grid.index.values] = net.ext_grid.max_p_mw
/var/folders/0t/9z26630n32l3qw5w80zvvj

#### Get edge index and attributes

In [8]:
branch = net.line[['from_bus', 'to_bus']]
branch

,from_bus,to_bus
0,0,1
1,0,2
2,3,4
3,2,4
4,4,5
5,5,6
6,7,8
7,8,9
8,3,10
9,4,10


In [65]:
branch_attr = net.line[['c_nf_per_km', 'r_ohm_per_km', 'x_ohm_per_km']]
branch_attr

,c_nf_per_km,r_ohm_per_km,x_ohm_per_km
0,353.789081,5.770332,19.024956
1,150.708577,2.456676,8.074656
2,29.250278,0.335174,1.519711
3,395.575193,4.589604,20.567520
4,198.623319,2.266236,10.283760
...,...,...,...
168,274.674042,3.123216,14.111604
169,38.443223,0.438012,1.980576
170,498.647602,6.265476,26.661600
171,166.865874,2.761380,9.160164


In [66]:
trafo = net.trafo[['hv_bus', 'lv_bus']]
trafo

,hv_bus,lv_bus
0,7,4
1,25,24
2,29,16
3,37,36
4,62,58
5,63,60
6,64,65
7,64,67
8,67,68
9,80,67


In [67]:
trafo_attr = net.trafo[['sn_mva', 'tap_neutral', 'tap_pos', 'tap_step_percent', 'vk_percent']]
trafo_attr

,sn_mva,tap_neutral,tap_pos,tap_step_percent,vk_percent
0,9900.0,0.0,-1.0,1.5,264.330000
1,9900.0,0.0,-1.0,4.0,378.180000
2,9900.0,0.0,-1.0,4.0,384.120000
3,9900.0,0.0,-1.0,6.5,371.250000
4,9900.0,0.0,-1.0,4.0,382.140000
5,9900.0,0.0,-1.0,1.5,265.320000
6,9900.0,0.0,-1.0,6.5,366.300000
7,9900.0,NaN,NaN,NaN,158.988082
8,9900.0,0.0,-1.0,6.5,366.300000
9,9900.0,NaN,NaN,NaN,200.729061


#### Save node and edge informartion

In [68]:
network.to_csv('network/node_features.csv')
branch.to_csv('network/branch_index.csv')
branch_attr.to_csv('network/branch_attr.csv')
trafo.to_csv('network/trafo_index.csv')
trafo_attr.to_csv('network/trafo_attr.csv')

#### Save normalized data and scaler

In [69]:
from sklearn.preprocessing import MinMaxScaler
import joblib

# Normalize branch atrributes
branch_attr_scaler = MinMaxScaler()
branch_attr_scaled = branch_attr_scaler.fit_transform(branch_attr)
pd.DataFrame(branch_attr_scaled).to_csv('network/branch_attr_scaled.csv')
joblib.dump(branch_attr_scaler, 'network/branch_attr_scaler.scaler')

# Normalize transformer attributes
trafo_attr_scaler = MinMaxScaler()
trafo_attr_scaled = trafo_attr_scaler.fit_transform(trafo_attr)
pd.DataFrame(trafo_attr_scaled).to_csv('network/trafo_attr_scaled.csv')
joblib.dump(trafo_attr_scaler, 'network/trafo_attr_scaler.scaler')

['network/trafo_attr_scaler.scaler']